In [1]:
from notebook.services.config import ConfigManager

# Get the notebook configuration manager
cm = ConfigManager()

# Update RISE configuration with explicit width and height
config_data = {
    'width': '100%',      # Use percentage or pixel values like 1600
    'height': '100%',     # Use percentage or pixel values like 900
    'margin': 0.05,
    'minScale': 0.2,
    'maxScale': 2.0,
    'theme': 'simple',
    'transition': 'slide'
}

result = cm.update('livereveal', config_data)
print("RISE configuration updated:", result)

RISE configuration updated: {'theme': 'simple', 'transition': 'slide', 'width': '100%', 'height': '100%', 'margin': 0.05, 'minScale': 0.2, 'maxScale': 2.0}


In [2]:
%%html
<link rel="stylesheet" href="custom.css">


<div style="text-align: center;">

<div style="background: white; display: inline-block; padding: 15px 90px; margin-bottom: 20px;">
<img src="images/seekcommons_logo.png" style="height: 60px;">
</div>

<div style="background-color: #5f7783; padding: 60px 40px; color: white; font-family: Arial, sans-serif;">

<h1 style="color: white; font-size: 56px; margin: 30px 0 20px 0; font-weight: 300;">KG accessibility for mortals: query objects (no SPARQL)</h1>

<h2 style="color: white; font-size: 28px; margin: 20px 0 30px 0; font-weight: 300;">Wolfgang Fahl</h2>

<div style="background: white; color: black; display: inline-block; padding: 15px 40px; margin: 20px 0; font-size: 22px;">
<strong>SEEKCommons</strong> Workshop: Build & Explore the Resource Hub
</div>

<h2 style="color: white; font-size: 36px; margin: 40px 0; font-weight: 300;">2025-11-17</h2>

</div>

<div style="background: white; display: inline-block; padding: 20px 40px; margin-top: 20px;">
<img src="images/nsf_logo.png" style="width: 200px;">
</div>

</div>

# KG Accessibility for Mortals: Query Objects

| **Concept** | **Details** |
|-------------|-------------|
| **What Are Query Objects?** | Pre-built, parameterized queries<br>No SPARQL required<br>Safe, constrained access for non-experts |
| **How They Work** | Trigger via simple HTTP endpoint / URL<br>Parameters define scope, filters, and outputs<br>Results returned as JSON or CSV |
| **Why They Matter** | Users can "ask the graph" without understanding the schema<br>Enables Sheets, notebooks, dashboards, and apps to consume graph data easily<br>Decouples tooling from SPARQL and internal model complexity |
| **Typical Use Cases** | Dashboards pulling live graph data<br>Data validation views<br>Simple reports for domain experts |


# What Are Query Objects?

https://cr.bitplan.com

<div style="overflow: auto;">
<img src="images/ListOfQueries.png" style="width: 45%; float: right; margin-left: 20px; margin-bottom: 10px;">

## Query Repository
- 49 pre-built queries ready to use
- Each query has: name, title, tryit link, SPARQL link
- No SPARQL knowledge needed
- Examples: AllPapers, AllScholars, AllInstitutions

</div>

# Selecting from a Query Set
https://snapquery.bitplan.com/

<div style="overflow: auto;">
<img src="images/SnapQueryAuthorListOfPublications.png" style="width: 45%; float: right; margin-left: 20px; margin-bottom: 10px;">

## Query Sets
structured via
- domain
- namespace
- name
- url

The example is from the Scholia Project
</div>


# Running a Query via Web Interface
<div style="text-align: center;">
<img src="images/ScholiaQueryAuthorListOfPublicationsQ80.png" style="max-width: 90%; height: auto;">
</div>


# Query Result
<div style="text-align: center;">
<img src="images/ScholiaQueryAuthorListOfPublicationsQ80Result.png" style="max-width: 90%; height: auto;">
</div>


# SeekCommons use case
https://raw.githubusercontent.com/SEEKCommons/resource-hub/refs/heads/main/documentation/query_objects.yaml
    
```yaml

'WhatLinksToSEEKCommons':
  # Finds all Wikidata entities linking to the SEEKCommons project (Q118147033)
  param_list:
    - name: limit
      type: int
      default_value: 50
  sparql: |
    SELECT ?item ?itemLabel ?property ?propertyLabel
    WHERE {
      ?item ?property wd:Q118147033 .
      ?prop wikibase:directClaim ?property .
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
    }
    LIMIT {{ limit }}
```

## Running Query Set directly in Jupyter notebook
224 lines of python code in query_set_runner.py created with Claude AI
```python
# Interactive SparqlQuerySet Runner
# Author: Wolfgang Fahl / bitplan.com
# Runs queries from YAML configuration files using SPARQL
# WF 2025-11-16
import requests
import yaml
from lodstorage.sparql import SPARQL
from lodstorage.query import Query, QueryManager
from typing import Optional, Dict, List
import sys


class QuerySetRunner:
    """
    Runs queries from a YAML configuration file using specified language e.g. SPARQL
    """

    def __init__(self, yaml_url: str, endpoints: Dict[str, Dict] = None):
        """
        Initialize with YAML configuration URL.

        Args:
            yaml_url: URL to the YAML file containing query definitions
            endpoints: Dictionary of endpoint configurations
        """
        self.yaml_url = yaml_url
        self.config = None
        self.query_manager = None
        self.endpoints = endpoints or {}
```

In [3]:
%run query_set_runner.py

Dropdown(description='Endpoint:', options=('wikidata',), style=DescriptionStyle(description_width='initial'), …

Dropdown(description='Query:', options=('WhatLinksToSEEKCommons', 'SEEKCommonersWithORCIDs', 'SEEKCommonsWorkh…

IntText(value=10, description='Limit:')

Button(button_style='success', description='Run Query', style=ButtonStyle())

Output()

# Running via command Line
```bash
sparqlquery -qp query_objects.yaml -qn WhatLinksToSEEKCommons --params limit=3
```
```
json[
  {
    "item": "http://www.wikidata.org/entity/Q7183255",
    "itemLabel": "Philip Bourne",
    "property": "http://www.wikidata.org/prop/direct/P1416",
    "propertyLabel": "http://www.wikidata.org/prop/direct/P1416"
  },
  {
    "item": "http://www.wikidata.org/entity/Q20895785",
    "itemLabel": "Daniel Mietchen",
    "property": "http://www.wikidata.org/prop/direct/P1416",
    "propertyLabel": "http://www.wikidata.org/prop/direct/P1416"
  },
  {
    "item": "http://www.wikidata.org/entity/Q23016934",
    "itemLabel": "Liz Henry",
    "property": "http://www.wikidata.org/prop/direct/P1416",
    "propertyLabel": "http://www.wikidata.org/prop/direct/P1416"
  }
]
```

# How to create Queries

| **Approach** | **Details** |
|--------------|-------------|
| **Copy & Adapt** | Browse existing query catalogs:<br>• [Scholia queries](https://scholia.toolforge.org/)<br>• [Wikidata Examples](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)<br>• Use [SnapQuery](https://snapquery.bitplan.com/) to explore and modify |
| **Ask an Expert** | Reach out to the community:<br>• Lane Rasberry<br>• Daniel Mietchen<br>• Wolfgang Fahl<br>• SPARQL community forums |
| **Use Tools** | Interactive tools make learning easier:<br>• [wdgrid](https://wdgrid.bitplan.com/)<br>• Wikidata Query Service<br>• Query builders and visualizers |
| **Use LLMs** | AI assistants can help:<br>• ChatGPT<br>• Claude<br>• Ask: "Convert this to SPARQL..."<br>• Request explanations of existing queries |

# wdgrid tool: Research Grant Example

<div style="text-align: center;">
<a href="https://wd.grid.bitplan.com" target="_blank">
<img src="images/WdGridResearchGrant.png" alt="Research Grant Instances in Wikidata" style="max-width: 90%; height: auto;">
</a>
</div>

[Visit: https://wd.grid.bitplan.com](https://wd.grid.bitplan.com)

## wdgrid tool: Research Grant Truly Tabular Analysis

<div style="text-align: center;">
<a href="https://wdgrid.bitplan.com/tt/Q54875403" target="_blank">
<img src="images/WdGridResearchGrantTT.png" alt="Research Grant Type Tree" style="max-width: 90%; height: auto;">
</a>
</div>

[Visit: https://wdgrid.bitplan.com/tt/Q54875403](https://wdgrid.bitplan.com/tt/Q54875403)

## Naive Query: Duplicate Row Problem

<div style="float: right; width: 50%;">
<img src="images/WdGridResearchGrantTTNaiveQuery.png" alt="Naive Query with Duplicate Rows" style="max-width: 100%; height: auto;">
</div>

<div style="width: 45%;">

**Issue:** Same research grant appears multiple times due to SPARQL 'solution approach'
- 7562 results but contains duplicates

[Visit: https://w.wiki/G8co](https://w.wiki/G8co) 
for a simplified version of the query

See my paper in CEUR-WS Volume 3262 http://ceurspt.wikidata.dbis.rwth-aachen.de/Vol-3262.html 3rd Wikidata Workshop 2022
co-located with the 21st International Semantic Web Conference (ISWC2022)

</div>

<div style="clear: both;"></div>
    

# What do you think?

# References